In [2]:
from model_implementation import nested_unet, attention_unet


In [5]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize

# Define function to load images and masks
def load_data(data_dir):
    images = []
    masks = []

    for subdir, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.tif'):
                # Check if the file is a mask
                if '_mask' in file:
                    mask_path = os.path.join(subdir, file)
                    mask = imread(mask_path)
                    masks.append(mask)
                else:
                    image_path = os.path.join(subdir, file)
                    image = imread(image_path)
                    images.append(image)

    # Convert lists to numpy arrays
    X = np.array(images)
    y = np.array(masks)

    # Resize images and masks if necessary
    X_resized = np.array([resize(image, (256, 256), mode='reflect') for image in X])
    y_resized = np.array([resize(mask, (256, 256), mode='reflect') for mask in y])

    return X_resized, y_resized

# Load your preprocessed data
data_directory = r"C:\Users\USER\Desktop\5Ctech\Data"
X, y = load_data(data_directory)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape for model input
X_train = X_train[..., np.newaxis]  # Add channel dimension
X_val = X_val[..., np.newaxis]
y_train = y_train[..., np.newaxis]
y_val = y_val[..., np.newaxis]


# Define models
input_shape = (256, 256, 1)
unet_model = nested_unet(input_shape=input_shape)
attention_model = attention_unet(input_shape=input_shape)

# Compile models
unet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
attention_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Checkpoints
checkpoint_unet = ModelCheckpoint('unet_model.h5', monitor='val_loss', save_best_only=True)
checkpoint_attention = ModelCheckpoint('attention_model.h5', monitor='val_loss', save_best_only=True)

# Train models
history_unet = unet_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                               epochs=50, batch_size=16, callbacks=[checkpoint_unet])

history_attention = attention_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                                         epochs=50, batch_size=16, callbacks=[checkpoint_attention])

# Evaluate models
def evaluate_model(model, X_val, y_val):
    predictions = model.predict(X_val)
    predictions = (predictions > 0.5).astype(np.uint8)  # Binarize predictions
    jaccard_index = jaccard_score(y_val.flatten(), predictions.flatten(), average='macro')
    return jaccard_index

unet_jaccard = evaluate_model(unet_model, X_val, y_val)
attention_jaccard = evaluate_model(attention_model, X_val, y_val)

print(f"U-Net Jaccard Index: {unet_jaccard}")
print(f"Attention U-Net Jaccard Index: {attention_jaccard}")

# Plot training history
def plot_history(history, title):
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    plt.show()

plot_history(history_unet, 'U-Net Training History')
plot_history(history_attention, 'Attention U-Net Training History')

# Save models after training
unet_model.save('final_unet_model.h5')
attention_model.save('final_attention_model.h5')


MemoryError: Unable to allocate 1.50 MiB for an array with shape (256, 256, 3) and data type float64